# Tutorial 8: Adding Custom Devices

**Goal:** Create custom device agents with domain-specific behavior.

**Time:** ~15 minutes

---

## Why Custom Devices?

HERON's `DeviceAgent` class (extending `FieldAgent`) provides:
- **Standardized lifecycle**: reset → observe → act → update
- **Automatic registration**: Devices integrate with environments seamlessly
- **Feature-based observation**: Use `FeatureProvider` for observable state
- **Action abstraction**: Define action spaces declaratively

## Step 1: Device Agent Lifecycle

Every device agent follows this lifecycle:

```
┌─────────────────────────────────────────────────────────┐
│                    Device Lifecycle                     │
├─────────────────────────────────────────────────────────┤
│  __init__()    → Configure device parameters            │
│       ↓                                                 │
│  reset()       → Initialize state for new episode       │
│       ↓                                                 │
│  ┌─────────────────────────────────────┐               │
│  │ STEP LOOP:                          │               │
│  │  observe()      → Get features      │               │
│  │       ↓                             │               │
│  │  act(action)    → Apply control     │               │
│  │       ↓                             │               │
│  │  update_state() → Physics update    │               │
│  │       ↓                             │               │
│  │  update_cost_safety() → Metrics     │               │
│  └─────────────────────────────────────┘               │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import ClassVar, Sequence, List, Dict, Any, Optional
from gymnasium import spaces

# In practice, import from heron and powergrid:
# from heron.core.feature import FeatureProvider
# from heron.agents.field_agent import FieldAgent
# from powergrid.agents.device_agent import DeviceAgent

print("Imports ready!")

## Step 2: Define Custom Feature

Features use the HERON `FeatureProvider` pattern with:
- `@dataclass(slots=True)` for memory efficiency
- `ClassVar[Sequence[str]]` for visibility (who can see the feature)
- `vector()` method returning numpy array for observation
- `names()` method for feature names

**Visibility levels:**
- `'owner'`: Only the device itself
- `'upper_level'`: Parent coordinator can see
- `'system'`: All coordinators can see
- `'public'`: All agents can see

In [ ]:
@dataclass(slots=True)
class WindTurbineFeature:
    """Custom feature for wind turbine state.
    
    Uses ClassVar[Sequence[str]] for visibility to specify who can observe.
    Uses slots=True for memory efficiency (important for large-scale simulations).
    """
    # Class-level: visibility determines who can observe this feature
    visibility: ClassVar[Sequence[str]] = ('owner', 'upper_level')
    feature_name: ClassVar[str] = 'WindTurbineFeature'
    
    # Device parameters (set at init, don't change during episode)
    p_rated_MW: float = 2.0      # Rated power capacity
    cut_in_speed: float = 3.0    # Minimum wind speed (m/s)
    rated_speed: float = 12.0    # Wind speed at rated power (m/s)
    cut_out_speed: float = 25.0  # Maximum safe wind speed (m/s)
    
    # Dynamic state (changes each timestep)
    wind_speed_ms: float = 8.0   # Current wind speed
    curtailment: float = 1.0     # Curtailment factor (0-1)
    p_output_MW: float = 0.0     # Current power output
    
    def vector(self) -> np.ndarray:
        """Convert to observation vector (normalized for RL)."""
        return np.array([
            self.wind_speed_ms / self.rated_speed,  # Normalized wind
            self.curtailment,                        # Already 0-1
            self.p_output_MW / self.p_rated_MW,     # Normalized power
        ], dtype=np.float32)
    
    def names(self) -> List[str]:
        """Feature names for debugging and logging."""
        return ['wind_speed_norm', 'curtailment', 'power_output_norm']
    
    def to_dict(self) -> Dict[str, float]:
        """Serialize to dict for state storage."""
        return {
            'p_rated_MW': self.p_rated_MW,
            'wind_speed_ms': self.wind_speed_ms,
            'curtailment': self.curtailment,
            'p_output_MW': self.p_output_MW,
        }


# Test the feature
feature = WindTurbineFeature(wind_speed_ms=10.0, p_output_MW=1.5)
print(f"Feature vector: {feature.vector()}")
print(f"Feature names: {feature.names()}")
print(f"Visibility: {feature.visibility}")
print(f"Feature name: {feature.feature_name}")

## Step 3: Implement Device Agent

A custom device agent (extending FieldAgent/DeviceAgent) must implement:

1. **`__init__()`**: Initialize with features list
2. **`init_action()`**: Define the action space
3. **`set_action(action)`**: Apply action to internal state
4. **`set_state(state_dict)`**: Update state from environment
5. **`apply_action(scheduler)`**: Schedule action effects
6. **`compute_local_reward()`**: Calculate reward/cost/safety

In [ ]:
@dataclass
class WindTurbineConfig:
    """Configuration for wind turbine."""
    bus: str
    p_rated_MW: float = 2.0
    cut_in_speed: float = 3.0
    rated_speed: float = 12.0
    cut_out_speed: float = 25.0


class WindTurbine:
    """Wind turbine device agent.
    
    In practice, this would extend DeviceAgent from powergrid:
        from powergrid.agents.device_agent import DeviceAgent
        class WindTurbine(DeviceAgent): ...
    
    Action: Curtailment factor [0, 1]
    - 0 = Fully curtailed (no power)
    - 1 = Maximum available power
    
    Physics:
    - Power depends on wind speed (cubic relationship)
    - Curtailment reduces output below available
    """
    
    def __init__(self, agent_id: str, config: Dict[str, Any]):
        """Initialize wind turbine.
        
        In the real implementation, you would call:
            super().__init__(
                agent_id=agent_id,
                features=[self._feature],  # Pass features to FieldAgent
            )
        """
        self.agent_id = agent_id
        self.config = WindTurbineConfig(**config)
        
        # Initialize feature (stored in features dict as {feature_name: feature})
        self._feature = WindTurbineFeature(
            p_rated_MW=self.config.p_rated_MW,
            cut_in_speed=self.config.cut_in_speed,
            rated_speed=self.config.rated_speed,
            cut_out_speed=self.config.cut_out_speed,
        )
        
        # Features dict (mimics FieldAgent.state.features)
        self.features = {self._feature.feature_name: self._feature}
        
        # Metrics for reward computation
        self.cost = 0.0
        self.safety = 0.0
        self.available_power = 0.0
    
    def init_action(self) -> spaces.Box:
        """Define action space: curtailment factor [0, 1].
        
        Required by FieldAgent interface.
        """
        return spaces.Box(
            low=np.array([0.0], dtype=np.float32),
            high=np.array([1.0], dtype=np.float32),
            dtype=np.float32,
        )
    
    def get_observation_space(self) -> spaces.Box:
        """Define observation space based on feature."""
        return spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(self._feature.vector()),),
            dtype=np.float32,
        )
    
    def set_state(self, state_dict: Dict[str, Any]) -> None:
        """Update state from environment.
        
        Called when environment pushes new state (e.g., wind speed from dataset).
        Required by FieldAgent interface.
        """
        if 'wind_speed' in state_dict:
            self._feature.wind_speed_ms = state_dict['wind_speed']
            self._compute_available_power()
    
    def set_action(self, action: np.ndarray) -> None:
        """Apply action (curtailment) to internal state.
        
        Required by FieldAgent interface.
        """
        self._feature.curtailment = float(np.clip(action[0], 0.0, 1.0))
    
    def apply_action(self, scheduler=None) -> None:
        """Apply action effects (update power output).
        
        In event-driven mode, this schedules action_effect events.
        Required by FieldAgent interface.
        """
        # Apply curtailment to available power
        self._feature.p_output_MW = self.available_power * self._feature.curtailment
    
    def _compute_available_power(self):
        """Compute available power from wind speed.
        
        Wind turbine power curve:
        - Below cut-in: 0
        - Cut-in to rated: Cubic relationship
        - Rated to cut-out: Constant at rated power
        - Above cut-out: 0 (safety shutdown)
        """
        v = self._feature.wind_speed_ms
        v_in = self.config.cut_in_speed
        v_rated = self.config.rated_speed
        v_out = self.config.cut_out_speed
        p_rated = self.config.p_rated_MW
        
        if v < v_in or v > v_out:
            self.available_power = 0.0
        elif v < v_rated:
            # Cubic relationship in the partial load region
            self.available_power = p_rated * ((v - v_in) / (v_rated - v_in)) ** 3
        else:
            # At rated power
            self.available_power = p_rated
    
    def observe(self) -> np.ndarray:
        """Get current observation vector."""
        return self._feature.vector()
    
    def compute_local_reward(self, local_state: Dict = None) -> float:
        """Calculate cost and safety metrics.
        
        Required by FieldAgent interface for reward computation.
        
        Cost: Opportunity cost of curtailment (lost revenue)
        Safety: Penalty for operating outside safe bounds
        """
        # Opportunity cost: curtailed power × price (simplified)
        curtailed_MW = self.available_power - self._feature.p_output_MW
        price_per_MWh = 30.0  # $/MWh
        self.cost = curtailed_MW * price_per_MWh
        
        # Safety: Penalty if wind speed near cut-out
        if self._feature.wind_speed_ms > self.config.cut_out_speed * 0.9:
            self.safety = 10.0  # Near cut-out warning
        else:
            self.safety = 0.0
        
        # Return negative cost as reward (minimizing cost = maximizing reward)
        return -(self.cost + self.safety)
    
    def reset(self, wind_speed: float = 8.0):
        """Reset for new episode."""
        self._feature.wind_speed_ms = wind_speed
        self._feature.curtailment = 1.0
        self._feature.p_output_MW = 0.0
        self.cost = 0.0
        self.safety = 0.0
        self._compute_available_power()


# Test the device
turbine = WindTurbine(
    agent_id='wind_1',
    config={'bus': 'Bus_1', 'p_rated_MW': 2.0}
)

print(f"Action space: {turbine.init_action()}")
print(f"Observation space: {turbine.get_observation_space()}")
print(f"Features: {list(turbine.features.keys())}")

## Step 4: Device Lifecycle Demo

Simulate the complete device lifecycle:

In [ ]:
# Simulate wind profile (hourly for one day)
np.random.seed(42)
hours = 24
base_wind = 8.0  # Base wind speed m/s
wind_profile = base_wind + 4 * np.sin(np.linspace(0, 2*np.pi, hours)) + np.random.randn(hours)
wind_profile = np.clip(wind_profile, 0, 30)

# Create turbine
turbine = WindTurbine('wind_1', {'bus': 'Bus_1', 'p_rated_MW': 2.0})

# Episode simulation
print("Wind Turbine Simulation")
print("=" * 60)

# Reset
turbine.reset(wind_speed=wind_profile[0])
print(f"Initial state: wind={wind_profile[0]:.1f} m/s")

total_energy = 0.0
total_reward = 0.0

for t in range(hours):
    # 1. Observe
    obs = turbine.observe()
    
    # 2. Policy decides action (simple heuristic: curtail in high wind)
    if wind_profile[t] > 20:  # Near cut-out, reduce load
        action = np.array([0.5])
    else:
        action = np.array([1.0])  # Full output
    
    # 3. Set action (FieldAgent.set_action)
    turbine.set_action(action)
    
    # 4. Set state from environment (FieldAgent.set_state)
    turbine.set_state({'wind_speed': wind_profile[t]})
    
    # 5. Apply action effects (FieldAgent.apply_action)
    turbine.apply_action()
    
    # 6. Compute reward (FieldAgent.compute_local_reward)
    reward = turbine.compute_local_reward()
    
    # Accumulate metrics
    total_energy += turbine._feature.p_output_MW  # MWh (1 hour timestep)
    total_reward += reward
    
    if t % 6 == 0:  # Print every 6 hours
        print(f"Hour {t:2d}: wind={wind_profile[t]:5.1f} m/s, "
              f"available={turbine.available_power:.2f} MW, "
              f"output={turbine._feature.p_output_MW:.2f} MW, "
              f"curtail={turbine._feature.curtailment:.1f}")

print("=" * 60)
print(f"Total energy: {total_energy:.1f} MWh")
print(f"Capacity factor: {total_energy / (hours * turbine.config.p_rated_MW) * 100:.1f}%")
print(f"Total reward: {total_reward:.2f}")

## Step 5: Device Registry Pattern

Register devices for factory-based creation (used in configuration-driven environments):

In [ ]:
# Device registry for factory pattern
DEVICE_REGISTRY: Dict[str, type] = {}


def register_device(name: str):
    """Decorator to register device classes."""
    def decorator(cls):
        DEVICE_REGISTRY[name] = cls
        return cls
    return decorator


def create_device(device_type: str, agent_id: str, config: Dict) -> Any:
    """Factory function to create devices from config."""
    if device_type not in DEVICE_REGISTRY:
        raise ValueError(f"Unknown device type: {device_type}")
    return DEVICE_REGISTRY[device_type](agent_id, config)


# Register our wind turbine
DEVICE_REGISTRY['WindTurbine'] = WindTurbine


# Example: Create from configuration
device_configs = [
    {'type': 'WindTurbine', 'name': 'wind_1', 'config': {'bus': 'Bus_1', 'p_rated_MW': 2.0}},
    {'type': 'WindTurbine', 'name': 'wind_2', 'config': {'bus': 'Bus_2', 'p_rated_MW': 3.0}},
]

devices = {}
for cfg in device_configs:
    device = create_device(cfg['type'], cfg['name'], cfg['config'])
    devices[cfg['name']] = device
    print(f"Created {cfg['type']}: {device.agent_id} ({device.config.p_rated_MW} MW)")

print(f"\nRegistered devices: {list(DEVICE_REGISTRY.keys())}")

## Step 6: Adding Another Device Type

Let's add a Solar Panel device to demonstrate the pattern:

In [ ]:
@dataclass(slots=True)
class SolarPanelFeature:
    """Feature for solar panel using ClassVar visibility pattern."""
    visibility: ClassVar[Sequence[str]] = ('owner', 'upper_level')
    feature_name: ClassVar[str] = 'SolarPanelFeature'
    
    p_rated_kW: float = 100.0
    irradiance: float = 0.0      # 0-1 normalized
    temperature_c: float = 25.0
    curtailment: float = 1.0
    p_output_kW: float = 0.0
    
    def vector(self) -> np.ndarray:
        return np.array([
            self.irradiance,
            self.temperature_c / 50.0,  # Normalize
            self.curtailment,
            self.p_output_kW / self.p_rated_kW,
        ], dtype=np.float32)
    
    def names(self) -> List[str]:
        return ['irradiance', 'temperature_norm', 'curtailment', 'power_output_norm']


class SolarPanel:
    """Solar panel with irradiance-based generation.
    
    Action: Curtailment factor [0, 1]
    Physics: P = irradiance × rated × efficiency × (1 - temp_derating) × curtailment
    """
    
    def __init__(self, agent_id: str, config: Dict[str, Any]):
        self.agent_id = agent_id
        self.p_rated_kW = config.get('p_rated_kW', 100.0)
        self.efficiency = config.get('efficiency', 0.20)
        self.temp_coef = config.get('temp_coef', -0.004)  # -0.4%/°C above 25°C
        
        self._feature = SolarPanelFeature(p_rated_kW=self.p_rated_kW)
        self.features = {self._feature.feature_name: self._feature}
        self.cost = 0.0
        self.safety = 0.0
    
    def init_action(self) -> spaces.Box:
        return spaces.Box(
            low=np.array([0.0], dtype=np.float32),
            high=np.array([1.0], dtype=np.float32),
        )
    
    def reset(self):
        self._feature.irradiance = 0.0
        self._feature.temperature_c = 25.0
        self._feature.curtailment = 1.0
        self._feature.p_output_kW = 0.0
    
    def observe(self) -> np.ndarray:
        return self._feature.vector()
    
    def set_action(self, action: np.ndarray):
        self._feature.curtailment = float(np.clip(action[0], 0.0, 1.0))
    
    def set_state(self, state_dict: Dict[str, Any]):
        """Update state with environmental conditions."""
        if 'irradiance' in state_dict:
            self._feature.irradiance = state_dict['irradiance']
        if 'temperature' in state_dict:
            self._feature.temperature_c = state_dict['temperature']
    
    def apply_action(self, scheduler=None):
        """Apply action and compute power output."""
        # Temperature derating
        temp_factor = 1.0 + self.temp_coef * (self._feature.temperature_c - 25.0)
        temp_factor = np.clip(temp_factor, 0.5, 1.1)
        
        # Power output
        available = self.p_rated_kW * self._feature.irradiance * temp_factor
        self._feature.p_output_kW = available * self._feature.curtailment
    
    def compute_local_reward(self, local_state: Dict = None) -> float:
        # Simplified: zero cost for renewables, maximize output
        self.cost = 0.0
        self.safety = 0.0
        return self._feature.p_output_kW / self.p_rated_kW  # Normalized reward


# Register solar panel
DEVICE_REGISTRY['SolarPanel'] = SolarPanel

# Test
solar = create_device('SolarPanel', 'solar_1', {'p_rated_kW': 100})
solar.reset()
solar.set_action(np.array([1.0]))
solar.set_state({'irradiance': 0.8, 'temperature': 35.0})
solar.apply_action()

print(f"Solar panel at 80% irradiance, 35°C:")
print(f"  Output: {solar._feature.p_output_kW:.1f} kW / {solar.p_rated_kW} kW rated")
print(f"  Observation: {solar.observe()}")
print(f"  Features: {list(solar.features.keys())}")

## Step 7: Integration with PowerGridAgent

Devices integrate with the coordinator (PowerGridAgent) as subordinates:

In [ ]:
class SimpleMicrogridCoordinator:
    """Simplified coordinator managing multiple devices.
    
    In practice, use PowerGridAgent:
        from powergrid.agents import PowerGridAgent
        microgrid = PowerGridAgent(
            agent_id='mg_1',
            subordinates=devices,  # Dict[AgentID, DeviceAgent]
        )
    """
    
    def __init__(self, coord_id: str, device_configs: List[Dict]):
        self.coord_id = coord_id
        self.devices: Dict[str, Any] = {}
        
        # Create devices from config (bottom-up building)
        for cfg in device_configs:
            device = create_device(cfg['type'], cfg['name'], cfg.get('config', {}))
            self.devices[cfg['name']] = device
    
    def reset(self):
        """Reset all devices."""
        for device in self.devices.values():
            device.reset()
    
    def get_total_power(self) -> float:
        """Sum power from all devices (aggregation)."""
        total = 0.0
        for device in self.devices.values():
            if hasattr(device._feature, 'p_output_MW'):
                total += device._feature.p_output_MW
            elif hasattr(device._feature, 'p_output_kW'):
                total += device._feature.p_output_kW / 1000.0  # Convert to MW
        return total
    
    def step(self, actions: Dict[str, np.ndarray], env_state: Dict):
        """Step all devices with actions and environment state."""
        total_reward = 0.0
        
        for name, device in self.devices.items():
            # 1. Set action
            if name in actions:
                device.set_action(actions[name])
            
            # 2. Set state from environment
            if isinstance(device, WindTurbine):
                device.set_state({'wind_speed': env_state.get('wind_speed', 8.0)})
            elif isinstance(device, SolarPanel):
                device.set_state({
                    'irradiance': env_state.get('irradiance', 0.5),
                    'temperature': env_state.get('temperature', 25.0)
                })
            
            # 3. Apply action
            device.apply_action()
            
            # 4. Compute reward
            total_reward += device.compute_local_reward()
        
        return total_reward


# Create a microgrid with mixed devices
mg = SimpleMicrogridCoordinator('mg_1', [
    {'type': 'WindTurbine', 'name': 'wind_1', 'config': {'bus': 'B1', 'p_rated_MW': 2.0}},
    {'type': 'SolarPanel', 'name': 'solar_1', 'config': {'p_rated_kW': 500}},
])

mg.reset()

# Simulate one step
actions = {
    'wind_1': np.array([1.0]),   # Full wind output
    'solar_1': np.array([0.8]),  # Curtail solar to 80%
}

env_state = {
    'wind_speed': 10.0,    # Good wind
    'irradiance': 0.9,     # Sunny
    'temperature': 30.0,   # Warm
}

reward = mg.step(actions, env_state)

print(f"Microgrid {mg.coord_id} output:")
for name, device in mg.devices.items():
    if hasattr(device._feature, 'p_output_MW'):
        print(f"  {name}: {device._feature.p_output_MW:.2f} MW")
    elif hasattr(device._feature, 'p_output_kW'):
        print(f"  {name}: {device._feature.p_output_kW:.1f} kW")
print(f"  Total: {mg.get_total_power():.2f} MW")
print(f"  Combined reward: {reward:.2f}")

## Key Takeaways

1. **Feature Provider Pattern**
   ```python
   @dataclass(slots=True)
   class MyFeature:
       visibility: ClassVar[Sequence[str]] = ('owner', 'upper_level')
       feature_name: ClassVar[str] = 'MyFeature'
       
       # Instance fields...
       value: float = 0.0
       
       def vector(self) -> np.ndarray:
           return np.array([self.value], dtype=np.float32)
   ```

2. **Device Agent Structure** (extending FieldAgent/DeviceAgent)
   ```python
   class MyDevice(DeviceAgent):
       def __init__(self, agent_id, config):
           self._feature = MyFeature(...)
           super().__init__(agent_id=agent_id, features=[self._feature])
       
       def init_action(self) -> spaces.Box:         # Define action space
       def set_action(self, action):                # Apply action
       def set_state(self, state_dict):             # Update from env
       def apply_action(self, scheduler):           # Execute effects
       def compute_local_reward(self, state):       # Calculate reward
   ```

3. **Device Registry**
   ```python
   DEVICE_REGISTRY['MyDevice'] = MyDevice
   device = DEVICE_REGISTRY[device_type](agent_id, config)
   ```

4. **Lifecycle Methods**
   - `reset()`: Initialize state
   - `observe()`: Get observation vector
   - `set_action(action)`: Store action
   - `set_state(state_dict)`: Update from environment
   - `apply_action()`: Execute action effects
   - `compute_local_reward()`: Calculate cost/safety metrics

---

**Next:** Explore the full power grid case study in `powergrid/agents/` for production device implementations.